In [ ]:
import numpy as np
import gdspy
import matplotlib.pyplot as plt

%matplotlib inline
from functools import partial

In [ ]:
%load_ext autoreload
%autoreload 2
import snakenmake
from snakenmake import *

In [ ]:
def plot_cell(cell, exclude=(2,)):
    # FROM: https://github.com/heitzmann/gdspy/issues/42
    poly_dict = cell.get_polygons(by_spec=True)
    plt.figure(figsize=(20, 12))
    for layer_datatype, polys in poly_dict.items():
        if layer_datatype[0] in exclude:
            continue
        for poly in polys:
            plt.fill(*poly.T, lw=0.5, ec="k", fc=(1, 0, 0, 0.5))
    plt.axes().set_aspect("equal", "datalim")


def write_gds(main_cell, filename, unit=1.0e-6, precision=1.0e-9):
    writer = gdspy.GdsWriter(filename, unit=unit, precision=precision)
    for cell in [main_cell] + list(main_cell.get_dependencies(True)):
        writer.write_cell(cell)
    writer.close()

In [ ]:
coverslip_dims = np.array([55e3, 24e3])
chip_dims = np.array([23e3, 13e3])

In [ ]:
fov = 6.5 * 2048 / 40
fov

In [ ]:
23e3 * 13e3 / fov**2

# Text testing

In [ ]:
import text

In [ ]:
polygons, metrics = text.render_character("8")

In [ ]:
polygons

In [ ]:
a, b = polygons[:2]

In [ ]:
import scipy.spatial

In [ ]:
dists = scipy.spatial.distance.cdist(a, b)

In [ ]:
np.unravel_index(dists.argmin(), dists.shape)

In [ ]:
a[3]

In [ ]:
b[0]

In [ ]:
plt.scatter(*a.T, c="r")
plt.scatter(*b.T, c="g")
plt.scatter(*a[3], c="b")
plt.scatter(*b[0], c="b")

In [ ]:
combined = np.concatenate((a[:4], b[0:], (b[0],), a[3:]))
plt.plot(*combined.T)

# Test

In [ ]:
import snakenmake
from snakenmake import *

# main_cell = wafer([chip('Big Snake S3 FC90 L35 W1.5 LG20 TS2', snake, split=3, draw_trenches=False)]*6, "Big snakes\n180503")
# main_cell = chip('Big Snake S3 FC90 L35 W1.5 LG20 TS2', snake, split=3, draw_trenches=False)
main_cell = Cell("main")
main_cell.add(Text("foo_ij\n8Bbgopq", 100))
# main_cell.add(Text("8", 100))
plot_cell(main_cell)

In [ ]:
import freetype

In [ ]:
text.DEFAULT_FACE.set_char_size?

In [ ]:
freetype.FT_LOAD_NO_SCALE

In [ ]:
g.CellReference?

# Designs

In [ ]:
%%time
from snakenmake import *

metadata = {}
main_cell = wafer(
    [
        chip(
            "Big Snake S3 FC{fc} L35 W1.5 TS{ts} LG{lg}".format(fc=fc, ts=ts, lg=lg),
            snake,
            split=3,
            feeding_channel_width=fc,
            trench_spacing=ts,
            lane_gap=lg,
            draw_trenches=True,
            metadata=metadata,
        )
        for (fc, lg) in zip((90, 45), (20, 17))
        for ts in (1, 1.5, 2)
    ],
    "Big Snakes \n BS/JQS 180513",
)
write_gds(main_cell, "bigsnakes3.gds")

In [ ]:
metadata

In [ ]:
%%time
from snakenmake import *

metadata = {}
main_cell = wafer(
    [
        chip(
            "Big Snake S3 FC{fc} L35 W1.5 TS{ts} LG20".format(fc=fc, ts=ts),
            snake,
            split=3,
            feeding_channel_width=fc,
            trench_spacing=ts,
            draw_trenches=True,
            tick_labels=False,
            metadata=metadata,
        )
        for fc in (90,)
        for ts in (1, 1.5, 2)
    ],
    "Big Snakes Backup \n BS/JQS 180513",
)
write_gds(main_cell, "bigsnakes3backup.gds")